### 크롤링한 데이터 불러오기

In [ ]:
import pandas as pd
csv_result = pd.read_csv("./result_123.csv") 

In [ ]:
# 필요없는 column 제거
csv_result.drop('Unnamed: 0', axis=1, inplace=True)

### reply에 컴슬러가 여럿일 경우, 하나씩 나눠서 질문글 복제 ->  다른 df에 저장 -> 합치기

In [ ]:
import string
import numpy as np
from ast import literal_eval
f_list = []
for i in range(26):
    upper = string.ascii_uppercase[i]
    find = '컴슬러' + upper
    f_list.append(find)
new_list = []
# couns = []
one_reply=[]
for k in range(csv_result.shape[0]):
    dict_reply = literal_eval(csv_result.iloc[k,1])
    if str(dict_reply) == '{}':
        new_list.append(csv_result.iloc[k,0])
        one_reply.append('{}')
    else:
        len_reply = len(dict_reply)

        for m in range(len_reply):
            new_list.append(np.repeat(csv_result.iloc[k,0],len_reply)[m])

        for key, value in dict_reply.items():
            # couns.append(key)  # couns는 필요X
            one_reply.append(value)

one_reply2=[]
for q in range(len(one_reply)):
    one_string = one_reply[q][0] 
    for i in range(len(one_reply[q])-1): 
        one_string = one_string + one_reply[q][i+1]
    one_reply2.append(one_string)
new_df = pd.DataFrame(new_list)
reply_df = pd.DataFrame(one_reply2)

In [ ]:
df_all = pd.concat([new_df,reply_df], axis=1)
df_all.columns = ['concerns','reply']

### 특수문자, 필요없는 문자 등 제거

In [ ]:
csv_result['concerns'] = csv_result['concerns'].str.replace(pat=r'[\n\t\t\t\t\t\t]', repl=r"", regex=True)

In [ ]:
# 지우고 싶은 특수문자들 한번에 다 제거할 수 있는 코드

char="~-+=_\t\t\n'^@#$%&*()xa[]{}/;ㅋㅎㅡ人ㅜㅠ"  
for x in range(len(char)):
    df_all['concerns'] = df_all['concerns'].str.replace(char[x],r" ", regex=True)
    df_all['reply'] = df_all['reply'].str.replace(char[x],r" ", regex=True)
df_all

csv_result['concerns'] = csv_result['concerns'].str.replace(pat=r'[\n\t\t\t\t\t\t]', repl=r"", regex=True)
csv_result

### 특정 키워드 포함하는 문장 전체 제거
#### 키워드 들어간 문장 전체 제거 후, 온점(.)으로 잇기

* 키워드 : 국번없이 , 전화상담, 카카오톡, 문자상담, 사이버 상담, 채팅상담실, 24시간 (운영) 등

In [ ]:
import re
new_reply = []
#save_chat = []
for i in range(len(process1)): #행의 개수
    sentences = []
    sentences=re.findall(r"[\w' ]+", str(process1['reply'][i]))  # 한 cell의 글을 문장 단위로 끊어줌
    del_num = []
    for j in range(len(sentences)):
        each_sentence = sentences[j]
        if each_sentence.find("문자상담") > 0:  
            print(each_sentence)
            del_num.append(j)
        else:
            pass
    x = 0
    for k in range(len(del_num)):
        if k == 0:   # del_num에 1개 있을 때
            del sentences[del_num[k]]
        elif k > 0:  # del_num에 2개 이상 있을 때
            del sentences[del_num[k]-x]
        x = x + 1
    join_sen = '.'.join(sentences)
    new_reply.append(join_sen)
a = pd.DataFrame(new_reply)
process1 = pd.concat([concern_col,a], axis=1)
# 다시할 떈 process1.columns=['concerns', 'reply'] 추가!